In [ ]:
#main libraries
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
from scipy import stats

from math import log2 ,sqrt
#sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVR
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score,f1_score
from sklearn.tree import export_graphviz
from sklearn.model_selection import GridSearchCV

#from six import StringIO
#from IPython.display import Image
#from tqdm.notebook import tqdm_notebook as tqdm

from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error


In [3]:
print("fuck you" )

fuck you


In [4]:
data=pd.read_csv("adult.csv")
data

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
0,90,?,77053,HS-grad,9,Widowed,?,Not-in-family,White,Female,0,4356,40,United-States,<=50K
1,82,Private,132870,HS-grad,9,Widowed,Exec-managerial,Not-in-family,White,Female,0,4356,18,United-States,<=50K
2,66,?,186061,Some-college,10,Widowed,?,Unmarried,Black,Female,0,4356,40,United-States,<=50K
3,54,Private,140359,7th-8th,4,Divorced,Machine-op-inspct,Unmarried,White,Female,0,3900,40,United-States,<=50K
4,41,Private,264663,Some-college,10,Separated,Prof-specialty,Own-child,White,Female,0,3900,40,United-States,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,22,Private,310152,Some-college,10,Never-married,Protective-serv,Not-in-family,White,Male,0,0,40,United-States,<=50K
32557,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
32558,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
32559,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K


In [ ]:
#missing value
data.isin(['?']).sum()#missing data is from categorical data


In [ ]:
#unbalanced dataset
income = data['income'].value_counts(normalize=True)
round(income * 100, 2).astype('str') + ' %'

The dataset is imbalanced in many ways but here are few examples:

In [ ]:

import seaborn as sns
plt.style.use('seaborn-whitegrid')
plt.figure(figsize=(7, 5))
sns.barplot(x=data.income.index, y=data.income.values, palette='bright')
plt.title('Distribution of Income', fontdict={
          'fontname': 'Monospace', 'fontsize': 20, 'fontweight': 'bold'})
plt.xlabel('Income', fontdict={'fontname': 'Monospace', 'fontsize': 15})
plt.ylabel('Number of people', fontdict={
           'fontname': 'Monospace', 'fontsize': 15})
plt.tick_params(labelsize=10)
plt.show()


In [ ]:
# Creating a barplot for 'Sex'
sex = data['sex'].value_counts()

plt.style.use('default')
plt.figure(figsize=(7, 5))
sns.barplot(x=sex.index,y= sex.values)
plt.title('Distribution of Sex', fontdict={
          'fontname': 'Monospace', 'fontsize': 20, 'fontweight': 'bold'})
plt.xlabel('Sex', fontdict={'fontname': 'Monospace', 'fontsize': 15})
plt.ylabel('Number of people', fontdict={
           'fontname': 'Monospace', 'fontsize': 15})
plt.tick_params(labelsize=10)
plt.grid()
plt.show()

In [ ]:
import squarify
race = data['race'].value_counts()

plt.style.use('default')
plt.figure(figsize=(7, 5))
squarify.plot(sizes=race.values, label=race.index, value=race.values)
plt.title('Race distribution', fontdict={
          'fontname': 'Monospace', 'fontsize': 20, 'fontweight': 'bold'})
plt.show()

In [ ]:

le = LabelEncoder()
subset_data = data[['age', 'education.num', 'capital.gain','capital.loss','hours.per.week','income','fnlwgt']]
data['income'] = le.fit_transform(data['income'])
# Creating a pairplot of dataset
sns.pairplot(subset_data)

plt.show()

In [ ]:
corr = subset_data.corr()
mask = np.zeros_like(corr)
mask[np.triu_indices_from(mask)] = True
with sns.axes_style("white"):
    f, ax = plt.subplots(figsize=(7, 5))
    ax = sns.heatmap(corr, mask=mask, vmax=.3, square=True,
                     annot=True, cmap='RdYlGn')

plt.show()

prerocessing

In [ ]:

#fill missing values
data=data.replace('?',np.nan)#the dataset uses ? for missing data 
columns_with_nan = ['workclass', 'occupation', 'native.country']
for col in columns_with_nan:
    data[col].fillna(data[col].mode()[0], inplace=True)

In [ ]:
#label encoding
from sklearn.preprocessing import LabelEncoder
for col in data.columns:
    if data[col].dtypes == 'object':
        encoder = LabelEncoder()
        data[col] = encoder.fit_transform(data[col])

In [ ]:
X = data.drop('income', axis=1)
Y = data['income']

In [ ]:

X = X.drop(['workclass', 'education', 'race', 'sex',
            'capital.loss', 'native.country'], axis=1)#these features has the least importance according to extra trees classifier

In [ ]:
X.info()


SCALING

In [ ]:
for col in X.columns:
    scaler = StandardScaler()
    X[col] = scaler.fit_transform(X[col].values.reshape(-1, 1))

In [ ]:

from imblearn.over_sampling import RandomOverSampler
x = data.drop('income', axis=1)
Y = data['income']
ros = RandomOverSampler(random_state=42)
ros.fit(x, Y)
X, y = ros.fit_resample(x, Y)
round(y.value_counts(normalize=True) * 100, 2).astype('str') + ' %'

train test split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X,y, test_size=0.2, random_state=42)

In [ ]:
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("Y_train shape:", y_train.shape)
print("Y_test shape:", y_test.shape)

models

In [ ]:
#random forest
from sklearn.ensemble import RandomForestRegressor
rfr=RandomForestRegressor(random_state=42).fit(X_train,y_train)

In [ ]:
knn = KNeighborsClassifier().fit(X_train, y_train)

In [ ]:
from sklearn.svm import SVC
svc = SVC(random_state=42)
svc.fit(X_train, y_train)

In [ ]:
naiveB=GaussianNB().fit(X_train,y_train)

In [ ]:
DTree=DecisionTreeClassifier(random_state=42).fit(X_train,y_train)

In [ ]:
logR=LogisticRegression(random_state=42).fit(X_train,y_train)

In [ ]:
models =[logR,knn,DTree,naiveB,svc]
column = ["model name","accuracy score ","f1 score"]
scores=[]
def evaluateModel (models):
    for model in models:
        y_pred=model.predict(X_test)
        accScore=round(accuracy_score(y_test, y_pred) * 100, 2)
        f1Score=round(f1_score(y_test, y_pred) * 100, 2)
        model_scores = {"model name": type(model).__name__,
                        "accuracy score":accScore,
                        "f1 score": f1Score}
        scores.append(model_scores)
evaluateModel(models)
df=pd.DataFrame(scores,columns=column)
df
    

In [ ]:
#random forest is a classification algorithm and we cant get the accuracy score for that